In [5]:
# review -> 이모티콘/자음 삭제 -> (띄워쓰기 + 맞춤법) -> 문장 분리
# - 문장 단위: 문장 부호, 종결어미, 연결조사로 사전 만들기

In [6]:
import pandas as pd
import re
import itertools

from kiwipiepy import Kiwi
from emoji import core
from hanspell import spell_checker
from pykospacing import Spacing

In [7]:
data = pd.read_csv('./data/raw/review_data.csv', encoding='utf-8-sig')

In [8]:
reviews = data['review']

In [9]:
kiwi = Kiwi()

In [10]:
# 사용자 사전 추가
kiwi.add_user_word('맛집', 'NNG', 0)
kiwi.add_user_word('라떼', 'NNG', 0)
kiwi.add_user_word('디카페인', 'NNG', 0)
# kiwi.add_user_word('카페명', 'NNP', 0) # 카페 이름 리스트가 있으면 추가

True

In [11]:
# 정규표현식 컴파일 
noun = re.compile(r'^N') # 명사형 태그
sp_char = re.compile(r'^(SS)|SW|SP|SO') # 특수문자 태그

In [12]:
# 리뷰 문장 단위로 분리
def revToSent(review):
    tmp = []
    sentences = []
    preTag = ''
    preForm = ''
    sents = kiwi.split_into_sents(review, return_tokens=True, normalize_coda=True)
    for sent in sents:
        toks = sent.tokens
        for tok in toks:
            if bool(sp_char.match(preTag)) and bool(sp_char.match(tok.tag)):
                preTag = tok.tag
                preForm = tok.form
                continue
            
            elif bool(sp_char.match(tok.tag)):
                sentences.append(kiwi.join(tmp))
                tmp.clear()
                
                preTag = tok.tag
                preForm = tok.form
                continue
            
            condition = (preTag == 'EC') and (preForm in ['고', '지만', '은데'])
            if condition and bool(noun.match(tok.tag)):
                sentences.append(kiwi.join(tmp))
                tmp.clear()

            tmp.append(tok)
            preTag = tok.tag
            preForm = tok.form            
            
        connective = (preTag == 'EF') and (preForm in ['던데', '는데', '은데'])
        if not connective and len(tmp) > 0:
            sentences.append(kiwi.join(tmp))
            tmp.clear()
    
    return sentences

In [13]:
preprocess = []

for review in reviews:
    # print('원본 문장: ', review)
    review = core.replace_emoji(review, replace="")
    # print('특수 문자 제거: ', review)
    sents = revToSent(review)
    preprocess.append(sents)
    # print('문장 분리: ', sents)
    # print()    

In [14]:
preprocess = list(itertools.chain(*preprocess))

In [15]:
preprocess[:5]

['맛있어요',
 '만족도 최상의 커피 맛집',
 '찐 커피 맛집',
 '로마노 진짜 달달하고 맛있고',
 '아이스크림 라떼도 꼬소하고 달달하고 진짜 맛있어요!!']

In [20]:
spacing = Spacing()
final = []

for sent_i in preprocess:
    new_sent_i = spell_checker.check(sent_i).checked
    new_sent_i = spacing(new_sent_i)
    # print('원본 : ', sent_i)
    # print('맞춤법+띄어 : ', new_sent_i)
    # print()
    final.append(new_sent_i)

1/1 [==============================] - 0s 45ms/step


In [22]:
df = pd.DataFrame({'reviews': preprocess})
df.to_csv('./data/train_x.csv', index=False)

In [23]:
df = pd.DataFrame({'reviews': final})
df.to_csv('./data/train_x_spellcheck.csv', index=False)